In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_gpd_gdf, to_st_gdf
from electoralyze.region.region_abc import RegionABC

In [2]:
region.SA1_2021.metadata

SA1_2021,geometry
i64,binary
10102100701,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x97\x07\x00\x00\x1a\x11\x03\xf8\x83\xbcb@\x90\xb1\xebt\x81\x8bA\xc0K\x07\x81\xbe\x89\xbcb@\x98\xe7[Du\x8bA\xc02\xce\x161\x8f\xbcb@\xfc\xa6\x1d""…"
10102100702,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x02\x00\x00\x00E\x03\x00\x00Y\xdc\xe9\xa6~\xb7b@\x9e\xfb\xe0\x81\x0c\xafA\xc0)\x8b)\x05\xd5\xb7b@\xcb\xe0\xbd\xd2=\xafA\xc0\xa7T\xa3\xd7\x09\xb8b@\xe5%\xec""…"
10102100703,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00o\x00\x00\x00\x0f\xf0N\x9b\xf5\xb8b@\xd60\xf1\x91g\xb8A\xc0g\xd4_\x7f\xf5\xb8b@\x87\xca\xcc?b\xb8A\xc01\x1eIR\xf5\xb8b@?bY""…"
10102100704,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00]\x00\x00\x00\x06:\xf2\xa5\x84\xb9b@\xfe\xecyB\x82\xb8A\xc0JEcw\x87\xb9b@\xf6\x08c\x13O\xb8A\xc0iw\x18^\x88\xb9b@\xadX\xab""…"
10102100705,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00g\x00\x00\x00\xdc\xbc\x02\x88p\xb9b@\xfe\x05l#y\xb9A\xc0g\x10R\x1cr\xb9b@X.\x07\x8cy\xb9A\xc0\xc1\xfd\xda$s\xb9b@I{\xf7""…"
…,…
90104100406,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00\x07\x00\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x000\x00\x00\x00\xe2\x11J\xca\xa8\xfed@)\xbd\x94\x953\x01=\xc0\x82\xc7f\xa2\xa8\xfed@\x1d\x9e\xb6\xb67\x01=\xc0\x09M""…"
90104100407,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00\x09\x00\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0f\x00\x00\x00\x88\x8d\x9c\xebT\xfdd@\x06\xf8)\xe8t\x02=\xc0\xf2\x07t\xbdT\xfdd@*\x9e\x87^q\x02=\xc0\x9a\xb3""…"
90104100408,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00\x06\x00\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x00\xfb\x02\x00\x00\xe3!t\xf8\xd2\xfed@`O\xc8Hv\x12=\xc0\x89\xd7l\x0d\xd3\xfed@=\xa9\x9e\xb1u\x12=\xc0^\xb6""…"


In [25]:
import shutil
import tempfile

from shapely import wkt

region_json = {
    "region_b": ["A", "B", "C"],
    "region_b_name": ["Alpha", "Bravo", "Charlie"],
    "extra": ["1", "2", "3"],
    "geometry": [
        "POLYGON ((-1 -1, 0 1, 1 -1, -1 -1))",
        "POLYGON ((-1 -1, 0 1, -2 1, -1 -1))",
        "POLYGON ((1 -1, 0 1, 2 1, 1 -1))",
    ],
}

# Create the GeoDataFrame
gdf = pl.DataFrame(region_json).with_columns(geometry=st.from_wkt("geometry"))


# Create a temporary directory for the shapefile
temp_dir = tempfile.TemporaryDirectory(delete=False)
shapefile_path = f"{temp_dir.name}/data.shp"

# Write to a shapefile
gdf.pipe(to_gpd_gdf).to_file(shapefile_path)

/tmp/ipykernel_7087/1189847169.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.pipe(to_gpd_gdf).to_file(shapefile_path)
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'region_b_name' to 'region_b_n'
  ogr_write(


In [27]:
# Read back the shapefile with Pyogrio
read_gdf = pyogrio.read_dataframe(shapefile_path)

# Print the read GeoDataFrame
print(read_gdf)

  region_b region_b_n extra                             geometry
0        A      Alpha     1  POLYGON ((-1 -1, 0 1, 1 -1, -1 -1))
1        B      Bravo     2  POLYGON ((-1 -1, -2 1, 0 1, -1 -1))
2        C    Charlie     3     POLYGON ((1 -1, 0 1, 2 1, 1 -1))


In [28]:
from electoralyze.common.functools import classproperty


class RegionMocked(RegionABC):
    @classproperty
    def raw_geometry_file(self) -> str:
        return shapefile_path

In [29]:
RegionMocked.raw_geometry_file

'/tmp/tmphy8cxqy1/data.shp'

In [30]:
RegionMocked._get_geometry_raw().with_columns(st.geom("geometry").st.area())

proj_create: Cannot find proj.db
proj_create: no database context specified


Extracting...


region_b,region_b_n,extra,geometry
str,str,str,f64
"""A""","""Alpha""","""1""",2.0
"""B""","""Bravo""","""2""",2.0
"""C""","""Charlie""","""3""",2.0


In [5]:
RegionABC.metadata

In [13]:
shutil.rmtree(temp_dir.name)